In [7]:
import os
import requests
from bs4 import BeautifulSoup
import csv
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'Malgun Gothic'  # 한글 폰트 설정
plt.rcParams['axes.unicode_minus'] = False  # 음수 부호 깨짐 방지

# 'app_manager/data' 폴더가 없으면 생성
data_folder = os.path.join('app_manager', 'data')
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

# 주식 코드와 이름 딕셔너리
stocks = {
    '005380': '현대차',
    '000270': '기아',
    '247540': '에코프로비엠',
    '051910': 'LG화학',
    '001570': '금양',
    '096770': 'SK이노베이션',
    '373220': 'LG 에너지솔루션',
    '006400': '삼성SDI',
    '066570': 'LG전자',
    '066970': '엘앤에프',
    '003670': '포스코퓨처엠',
    '022100': '포스코DX',
    '001740': 'SK 네트웍스',
    '248070': '솔루엠',
    '286940': '롯데이노베이트'
}

# 기본 URL
base_url = "https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A"

# 모든 기업에 대해 영업이익 데이터 추출 및 그래프 그리기
for code, name in stocks.items():
    # URL 생성
    url = f"{base_url}{code}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701"

    # 웹 페이지 요청
    response = requests.get(url)
    response.encoding = 'utf-8'  # 인코딩 설정

    # 페이지 내용 파싱
    soup = BeautifulSoup(response.text, 'html.parser')

    # 테이블 12번 선택
    tables = soup.find_all('table')
    if len(tables) < 12:
        print(f"{name} ({code}): 테이블이 존재하지 않습니다.")
        continue
    
    table_12 = tables[11]  # 12번째 테이블 (인덱스 11)

    # 영업이익 데이터 추출 (4행) 및 연도 데이터 추출 (2행)
    rows = table_12.find_all('tr')
    earnings_data = []
    year_data = []

    # 2행에서 연도 데이터 추출
    if len(rows) >= 2:
        year_cols = rows[1].find_all('td')  # 2행 (인덱스 1)
        year_data = [col.text.strip() for col in year_cols]

    # 4행에서 영업이익 데이터 추출
    if len(rows) >= 4:  # 4행이 있는 경우
        earnings_cols = rows[3].find_all('td')  # 4행 (인덱스 3)
        earnings_data = [col.text.strip() for col in earnings_cols]  # 영업이익 데이터 저장

    # 연도 데이터 생성 (2019년부터 시작)
    start_year = 2019
    year_data = [str(start_year + i) for i in range(len(earnings_data))]

    # CSV 파일로 저장
    csv_filename = os.path.join(data_folder, f"{name}_재무제표.csv")
    with open(csv_filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['년도', '영업이익'])  # 헤더 추가
        for year, earnings in zip(year_data, earnings_data):
            writer.writerow([year, earnings])  # 연도와 영업이익 저장

    print(f"{csv_filename} 저장 완료.")

    # 그래프 그리기
    years = year_data
    earnings = []

    for earnings_value in earnings_data:
        earnings_value = earnings_value.replace(',', '').replace('억', '').strip()
        if earnings_value:
            try:
                earnings.append(int(earnings_value))
            except ValueError:
                earnings.append(0)
        else:
            earnings.append(0)

    # 막대그래프 그리기
    if years and earnings:
        plt.figure(figsize=(10, 6))

        # 막대 색상 설정
        colors = ['orange' for earning in earnings]
        for i in range(len(years)):
            if years[i] in ['2024', '2025', '2026']:
                colors[i] = (1, 0.647, 0, 0.5)  # 2024, 2025, 2026년은 반투명한 초록색 (R=0, G=1, B=0, A=0.5)

        bars = plt.bar(years, earnings, color=colors)

        # y=0인 부분에 점선 추가
        plt.axhline(y=0, color='gray', linewidth=1, linestyle='--')  # y=0에 회색 점선 추가

        plt.title(f'{name} 연도별 영업이익')
        plt.ylabel('영업이익 (억 원)')
        plt.xticks(rotation=45)
        plt.tight_layout()  # 레이아웃 자동 조정

        # 그래프 저장
        save_filename = os.path.join(data_folder, f"{name}_영업이익_그래프.png")
        plt.savefig(save_filename)
        plt.close()  # 현재 그래프 닫기

        print(f"{save_filename} 저장 완료.")

print("모든 기업의 영업이익 데이터 저장 및 그래프 생성이 완료되었습니다.")


app_manager\data\현대차_재무제표.csv 저장 완료.
app_manager\data\현대차_영업이익_그래프.png 저장 완료.
app_manager\data\기아_재무제표.csv 저장 완료.
app_manager\data\기아_영업이익_그래프.png 저장 완료.
app_manager\data\에코프로비엠_재무제표.csv 저장 완료.
app_manager\data\에코프로비엠_영업이익_그래프.png 저장 완료.
app_manager\data\LG화학_재무제표.csv 저장 완료.
app_manager\data\LG화학_영업이익_그래프.png 저장 완료.
app_manager\data\금양_재무제표.csv 저장 완료.
app_manager\data\금양_영업이익_그래프.png 저장 완료.
app_manager\data\SK이노베이션_재무제표.csv 저장 완료.
app_manager\data\SK이노베이션_영업이익_그래프.png 저장 완료.
app_manager\data\LG 에너지솔루션_재무제표.csv 저장 완료.
app_manager\data\LG 에너지솔루션_영업이익_그래프.png 저장 완료.
app_manager\data\삼성SDI_재무제표.csv 저장 완료.
app_manager\data\삼성SDI_영업이익_그래프.png 저장 완료.
app_manager\data\LG전자_재무제표.csv 저장 완료.
app_manager\data\LG전자_영업이익_그래프.png 저장 완료.
app_manager\data\엘앤에프_재무제표.csv 저장 완료.
app_manager\data\엘앤에프_영업이익_그래프.png 저장 완료.
app_manager\data\포스코퓨처엠_재무제표.csv 저장 완료.
app_manager\data\포스코퓨처엠_영업이익_그래프.png 저장 완료.
app_manager\data\포스코DX_재무제표.csv 저장 완료.
app_manager\data\포스코DX_영업이익_그래프.png 저장 완료.
app_manager\data\SK 네트